## Raw ideas

so here's a few quick ideas for a validation schema:

- data shouldn't be shuffled before training, cause it's essential to have the time series values in consecutive order
- currently I came up with three ideas for a schema:
    1. `1,2train+3valid` -> `1,2,3train+4valid` -> `1,2,3,4train+5valid` etc.

        **Advatages**:
        - the train set grows, probably helping to test model's robustness
        - we always validate on different parts of the training set

        **Disadvantages**:
        - computationally heavy. at some point our train will contain almost the entire dataset
    2. `1,2train+3valid` -> `2,3train+4valid` -> `3,4train+5valid`

        **Advantages**:
        - fixed train size, no computational power issues
        - we can somewhat efficiently use our dataset since we use each fold several times during different trainig sessions
        
        **Disadvantages**:
        - the fact that different sets overlap might be a problem (shouldn't be, but who knows)
    3.  `1,2train+3valid` -> `4,5train+6valid` -> `7,8train+9valid`

        **Advantages**:
        - the easiest one to implement, goes through each fold once only
        - it's guaranteed that there's no overlapping or data leaks
        
        **Disadvantages**:
        - data hugry in some sense, each fold is used only ones and each trainig session takes $f_{train} + 1$ folds, where $f_{train}$ is the amount of folds for the trainig set. this might be a problem assuming we have only 33 folds at maximum

model ideas:

- **decision tree-based algorithms (boosting, random forest)**:

    seems to be a good choice for the task since we have a lot of categorical features and the target is a discrete variable (which is typical for decision tree regression). at the same time, trees can be overfit easily, which can cause a huge loss in robustness and very inaccurate predictions on unseen data. ensembles will help to resolve this, but the problem of overfitting won't be gone completely. also, decision trees don't work well once something completely unseen shows up in input data, which might be a problem during future usage
- **auto-regressive models**:

    a time-series specific solution, that's supposed to do a good job at predicting future sales based on historical data. the main problems are the following: 
        1. I don't have any experience working with them (but we'll still try I guess)
        2. the data is still quite noisy, so using AR models will require some additional preprocessing 
- **RNNs**:

    one of the hardest models to implement, but at the same time a really powerfull (in theory) solution, that by definition fits the task idea quite well. RNNs can be really good when it comes to analyzing consecitive data (like text processing or time series) 


useful additional features:

- day of the week
- month
- year
- lagged values (look them up in `EDA`)
- ...


In [1]:
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore', category=FutureWarning)
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

/tmp/ipykernel_21229/4242659641.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Loading the data

the point is to load the data in a format that's actually going to be fed to the model

In [2]:
data = pd.read_csv('../data/processed_files/processed_data.csv')
data['date'] = pd.to_datetime(data['date'])

In [3]:
data.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_name', 'item_category_id', 'shop_name',
       'item_category_name'],
      dtype='object')

In [4]:
#TODO should be moved to a dedicated file once i get everything working

class FoldLoader:
    """
    Provides tools for loading time series data, processing it and dividing into folds.
    Is iterable

    Example:

    >>> loader = FoldLoader(your_data, 3, folding_mode='over')
    >>> for train, val in loader:
    >>>     # whatever we wanna do with the folds

    This creates a FoldLoader with overlap folding mode.
    After that, you can iterate through it using a for loop, where on each iteration
    you recieve training and validation sets insize a 2-element tuple
    """
    def __init__(self, data: pd.DataFrame,  n_train_folds: int, n_valid_folds: int = 1, folding_mode: str = 'sequence') -> None:
        """
        Constructs a FoldLoader object, sets its internal properties 
        (data source, training and validation sets size, folding mode)

        Keyword arguments:

        data -- a pandas dataframe containing the training data

        target_col -- the name of a column in `data` contains the target values

        n_train_folds -- the amount of months used to train the model.
        must be less or equal to the amount of months in `data` minus n_valid_folds. 
        if `folding_mode` is 'stack', only affects the train size on the first iteration

        n_valid_folds -- the amount of months used to validate the model.
        must be less or equal to the amount of months in `data` minus n_train_folds
        Default: 1

        folding_mode -- one of "sequence", "overlap", "stack"
        a string denoting the data splitting mode:
            - `sequence` for `1,2train+3valid` -> `4,5train+6valid` -> `7,8train+9valid`
            - `overlap` for `1,2train+3valid` -> `2,3train+4valid` -> `3,4train+5valid`
            - `stack` for `1,2train+3valid` -> `1,2,3train+4valid` -> `1,2,3,4train+5valid`
        Default: `sequence`
        """
        self.min_fold = data['date_block_num'].unique().min()
        self.max_fold = data['date_block_num'].unique().max()
        self.current_fold = self.min_fold
        self.train_size = n_train_folds
        self.val_size = n_valid_folds
        self.mode = folding_mode

        if (self.max_fold - self.min_fold + 1) < self.train_size + self.val_size:
            raise ValueError(
                f'too many folds in train and valid for the given data')
        tmp = pd.DataFrame()
        tmp['income'] = data['item_price'] * data['item_cnt_day']
        self.data = data[['date_block_num', 'shop_id',
                    'item_id', 'item_cnt_day']].join(tmp, how='inner')
        self.data = self.data.groupby(
            ['date_block_num', 'shop_id', 'item_id'], as_index=False).sum()

    @staticmethod
    def preprocess(data: pd.DataFrame, cat_features) -> pd.DataFrame:
        """
        Preprocesses the input data. this includes the following:
            - target encoding columns specified in `cat_features`
            - forming additional features based on EDA results

        Keyword arguments:

        data -- data to be processed
    
        cat_features -- list of column names present in `self.data` that will be
        target encoded

        Returns:

        pandas.DataFrame -- a new dataframe with the processed data 
        """
        raise NotImplementedError

    def _get_item_internal(self, start_idx) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        iterates through the dataset to form the training and validation sets
        for current iteration

        Keyword arguments:

        start_idx -- index of the fold (month) to start from

        Returns:

        tuple(train, val) -- training and vaidation sets in form of a pandas DataFrame
        """
        train = pd.DataFrame(columns=self.data.columns)
        val = pd.DataFrame(columns=self.data.columns)

        for idx in range(start_idx, start_idx + self.train_size):
            train = pd.concat(
                [train, self.data[self.data['date_block_num'] == idx]])
        start_idx += self.train_size
        for idx in range(start_idx, start_idx + self.val_size):
            val = pd.concat(
                [val, self.data[self.data['date_block_num'] == idx]])
        return (train, val)

    def __getitem__(self, idx) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        calls `_get_item_internal` to get training and validation sets.
        adjusts internal counters corresponding to the selected folding mode.
        Raises IndexError once the iteration is over

        Keyword arguments:

        idx -- not used in there since the data has to be consecutive.
        however, it's mandatory to let __getitem__ take an index as input, 
        so there it is

        Returns:

        tuple(train, val) -- training and vaidation sets in form of a pandas DataFrame
        """
        if (self.current_fold+self.train_size+self.val_size - 1) > self.max_fold:
            raise IndexError
        start_idx = self.current_fold
        if self.mode.startswith('seq'):
            result = self._get_item_internal(start_idx)
            self.current_fold += self.train_size+self.val_size
            return result
        elif self.mode.startswith('over'):
            result = self._get_item_internal(start_idx)
            self.current_fold += 1
            return result
        elif self.mode.startswith('stack'):
            result = self._get_item_internal(start_idx)
            self.train_size += 1
            return result

    def reset_folds(self) -> None:
        """
        sets the current fold counter to point to the first fold 
        of the dataset. this effectively means that the iteration will be started over
        """
        self.current_fold = self.min_fold

In [5]:
loader = FoldLoader(data, 20, folding_mode='overlap')
for train, val in loader:
    print(train.shape[0], val.shape[0])

1058444 39937
1035652 42090
1018239 45165
999872 58263
1003865 46267
997172 41060
982462 40074
964925 32493
939824 31822
920559 31630
901629 33316
883909 33240
851608 29343
828094 31050
